In [59]:
import geopandas as gpd
from dotenv import load_dotenv
import os
from pyarrow.fs import S3FileSystem
import folium

In [18]:
load_dotenv()

True

In [19]:
AWS_S3_BUCKET = os.getenv("MINIO_SILVER_BUCKET_NAME")
AWS_ACCESS_KEY_ID = os.getenv("MINIO_ROOT_USER")
AWS_SECRET_ACCESS_KEY = os.getenv("MINIO_ROOT_PASSWORD")
ENDPOINT_OVERRIDE = os.getenv("MINIO_ENDPOINT_URL")

In [20]:
s3fs = S3FileSystem(
    access_key=AWS_ACCESS_KEY_ID,
    secret_key=AWS_SECRET_ACCESS_KEY,
    endpoint_override=ENDPOINT_OVERRIDE,
    )

## Carregar dados do asset `setor_censitario_enriched`

In [21]:
def _get_s3_path_for(table_name:str, bucket_name:str=AWS_S3_BUCKET) -> str:

    s3_path = f'{bucket_name}/dagster/{table_name}.parquet'
    return s3_path

In [22]:
def load_parquet(table_name:str, bucket_name:str=AWS_S3_BUCKET) -> gpd.GeoDataFrame: 
            
    s3_path = _get_s3_path_for(table_name)
    
    gdf = gpd.read_parquet(s3_path, filesystem=s3fs)
    return gdf

In [23]:
gdf_setor = load_parquet('setor_censitario_enriched')

In [24]:
gdf_setor

,cd_original_setor_censitario,geometry,Cod_setor,Domicilio01_SP1_V001,Domicilio01_SP1_V002,Domicilio01_SP1_V012,Domicilio01_SP1_V013,Domicilio01_SP1_V014,Domicilio01_SP1_V017,Domicilio01_SP1_V018,Domicilio01_SP1_V019,Domicilio01_SP1_V020,Domicilio01_SP1_V021,Domicilio01_SP1_V035,Domicilio01_SP1_V039,Domicilio01_SP1_V040,Domicilio01_SP1_V041,missing,supressed
0,355030801000001,"POLYGON ((339816.876 7392593.185, 339762.085 7...",355030801000001,268.0,266.0,266.0,0.0,0.0,266.0,0.0,0.0,0.0,0.0,265.0,0.0,1.0,0.0,False,False
1,355030801000002,"POLYGON ((339898.791 7392663.965, 339890.991 7...",355030801000002,306.0,306.0,306.0,0.0,0.0,306.0,0.0,0.0,0.0,0.0,306.0,0.0,0.0,0.0,False,False
2,355030801000003,"POLYGON ((340010.027 7392443.461, 339999.011 7...",355030801000003,189.0,189.0,189.0,0.0,0.0,187.0,1.0,1.0,0.0,0.0,189.0,0.0,0.0,0.0,False,False
3,355030801000004,"POLYGON ((340035.826 7392138.185, 339960.781 7...",355030801000004,181.0,181.0,181.0,0.0,0.0,181.0,0.0,0.0,0.0,0.0,181.0,0.0,0.0,0.0,False,False
4,355030801000005,"POLYGON ((339655.092 7392153.506, 339647.532 7...",355030801000005,240.0,240.0,240.0,0.0,0.0,240.0,0.0,0.0,0.0,0.0,240.0,0.0,0.0,0.0,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18948,355030896000243,"POLYGON ((358263.250 7395587.106, 358263.354 7...",355030896000243,96.0,96.0,96.0,0.0,0.0,96.0,0.0,0.0,0.0,0.0,96.0,0.0,0.0,0.0,False,False
18949,355030896000244,"POLYGON ((358308.049 7395452.215, 358307.823 7...",355030896000244,54.0,54.0,54.0,0.0,0.0,52.0,0.0,0.0,0.0,2.0,54.0,0.0,0.0,0.0,False,False
18950,355030896000245,"POLYGON ((354654.416 7398105.552, 354676.451 7...",355030896000245,87.0,87.0,87.0,0.0,0.0,15.0,0.0,0.0,0.0,72.0,87.0,0.0,0.0,0.0,False,False
18951,355030896000246,"POLYGON ((354778.438 7398059.051, 354812.268 7...",355030896000246,38.0,38.0,38.0,0.0,0.0,18.0,0.0,0.0,0.0,20.0,38.0,0.0,0.0,0.0,False,False


## Classificar aleatoriamente os setores em 5 categorias, nas seguintes proporções

1. 10% dos setores como high-high;

2. 
10% dos setores como low-low

3. 
5% dos setores como high-lo

4. ;
5% dos setores como low-hi

5. h;
70% dos setores como non-significant.

In [45]:
gdf_setor_lisa = gdf_setor.copy()
gdf_setor_lisa['lisa_class'] = 'non-significant'
gdf_setor_lisa

,cd_original_setor_censitario,geometry,Cod_setor,Domicilio01_SP1_V001,Domicilio01_SP1_V002,Domicilio01_SP1_V012,Domicilio01_SP1_V013,Domicilio01_SP1_V014,Domicilio01_SP1_V017,Domicilio01_SP1_V018,Domicilio01_SP1_V019,Domicilio01_SP1_V020,Domicilio01_SP1_V021,Domicilio01_SP1_V035,Domicilio01_SP1_V039,Domicilio01_SP1_V040,Domicilio01_SP1_V041,missing,supressed,lisa_class
0,355030801000001,"POLYGON ((339816.876 7392593.185, 339762.085 7...",355030801000001,268.0,266.0,266.0,0.0,0.0,266.0,0.0,0.0,0.0,0.0,265.0,0.0,1.0,0.0,False,False,non-significant
1,355030801000002,"POLYGON ((339898.791 7392663.965, 339890.991 7...",355030801000002,306.0,306.0,306.0,0.0,0.0,306.0,0.0,0.0,0.0,0.0,306.0,0.0,0.0,0.0,False,False,non-significant
2,355030801000003,"POLYGON ((340010.027 7392443.461, 339999.011 7...",355030801000003,189.0,189.0,189.0,0.0,0.0,187.0,1.0,1.0,0.0,0.0,189.0,0.0,0.0,0.0,False,False,non-significant
3,355030801000004,"POLYGON ((340035.826 7392138.185, 339960.781 7...",355030801000004,181.0,181.0,181.0,0.0,0.0,181.0,0.0,0.0,0.0,0.0,181.0,0.0,0.0,0.0,False,False,non-significant
4,355030801000005,"POLYGON ((339655.092 7392153.506, 339647.532 7...",355030801000005,240.0,240.0,240.0,0.0,0.0,240.0,0.0,0.0,0.0,0.0,240.0,0.0,0.0,0.0,False,False,non-significant
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18948,355030896000243,"POLYGON ((358263.250 7395587.106, 358263.354 7...",355030896000243,96.0,96.0,96.0,0.0,0.0,96.0,0.0,0.0,0.0,0.0,96.0,0.0,0.0,0.0,False,False,non-significant
18949,355030896000244,"POLYGON ((358308.049 7395452.215, 358307.823 7...",355030896000244,54.0,54.0,54.0,0.0,0.0,52.0,0.0,0.0,0.0,2.0,54.0,0.0,0.0,0.0,False,False,non-significant
18950,355030896000245,"POLYGON ((354654.416 7398105.552, 354676.451 7...",355030896000245,87.0,87.0,87.0,0.0,0.0,15.0,0.0,0.0,0.0,72.0,87.0,0.0,0.0,0.0,False,False,non-significant
18951,355030896000246,"POLYGON ((354778.438 7398059.051, 354812.268 7...",355030896000246,38.0,38.0,38.0,0.0,0.0,18.0,0.0,0.0,0.0,20.0,38.0,0.0,0.0,0.0,False,False,non-significant


In [47]:
sample_pct = 0.1
sample = gdf_setor_lisa.sample(frac=sample_pct)
sample

,cd_original_setor_censitario,geometry,Cod_setor,Domicilio01_SP1_V001,Domicilio01_SP1_V002,Domicilio01_SP1_V012,Domicilio01_SP1_V013,Domicilio01_SP1_V014,Domicilio01_SP1_V017,Domicilio01_SP1_V018,Domicilio01_SP1_V019,Domicilio01_SP1_V020,Domicilio01_SP1_V021,Domicilio01_SP1_V035,Domicilio01_SP1_V039,Domicilio01_SP1_V040,Domicilio01_SP1_V041,missing,supressed,lisa_class
9096,355030844000213,"POLYGON ((356912.808 7401886.417, 356914.738 7...",None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,False,non-significant
6453,355030833000211,"POLYGON ((355086.048 7386961.385, 355076.385 7...",None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,False,non-significant
18880,355030896000175,"POLYGON ((358217.122 7395163.409, 358211.952 7...",355030896000175,210.0,209.0,209.0,0.0,0.0,207.0,0.0,0.0,0.0,1.0,209.0,0.0,0.0,0.0,False,False,non-significant
7606,355030838000131,"POLYGON ((331641.892 7384693.580, 331645.392 7...",355030838000131,283.0,283.0,241.0,4.0,0.0,216.0,0.0,0.0,0.0,61.0,282.0,0.0,1.0,0.0,False,False,non-significant
1659,355030813000153,"POLYGON ((330092.725 7405346.890, 330121.033 7...",355030813000153,89.0,89.0,62.0,0.0,0.0,16.0,1.0,1.0,0.0,68.0,89.0,0.0,0.0,0.0,False,False,non-significant
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9336,355030846000013,"POLYGON ((322896.495 7381262.244, 322911.108 7...",355030846000013,177.0,177.0,177.0,0.0,0.0,173.0,4.0,0.0,0.0,0.0,177.0,0.0,0.0,0.0,False,False,non-significant
16585,355030883000175,"POLYGON ((323851.395 7386188.581, 323852.180 7...",355030883000175,100.0,100.0,100.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,False,False,non-significant
16253,355030882000018,"POLYGON ((335565.666 7401792.321, 335578.392 7...",355030882000018,145.0,144.0,144.0,0.0,0.0,109.0,3.0,1.0,0.0,31.0,144.0,0.0,0.0,0.0,False,False,non-significant
1993,355030816000055,"POLYGON ((327309.293 7381424.482, 327320.132 7...",355030816000055,212.0,208.0,208.0,0.0,0.0,202.0,0.0,0.0,0.0,6.0,208.0,0.0,0.0,0.0,False,False,non-significant


In [48]:
sample_filter = gdf_setor_lisa['cd_original_setor_censitario'].isin(sample['cd_original_setor_censitario'])
gdf_setor_lisa.loc[sample_filter, 'lisa_class'] = 'high-high'
gdf_setor_lisa

,cd_original_setor_censitario,geometry,Cod_setor,Domicilio01_SP1_V001,Domicilio01_SP1_V002,Domicilio01_SP1_V012,Domicilio01_SP1_V013,Domicilio01_SP1_V014,Domicilio01_SP1_V017,Domicilio01_SP1_V018,Domicilio01_SP1_V019,Domicilio01_SP1_V020,Domicilio01_SP1_V021,Domicilio01_SP1_V035,Domicilio01_SP1_V039,Domicilio01_SP1_V040,Domicilio01_SP1_V041,missing,supressed,lisa_class
0,355030801000001,"POLYGON ((339816.876 7392593.185, 339762.085 7...",355030801000001,268.0,266.0,266.0,0.0,0.0,266.0,0.0,0.0,0.0,0.0,265.0,0.0,1.0,0.0,False,False,non-significant
1,355030801000002,"POLYGON ((339898.791 7392663.965, 339890.991 7...",355030801000002,306.0,306.0,306.0,0.0,0.0,306.0,0.0,0.0,0.0,0.0,306.0,0.0,0.0,0.0,False,False,non-significant
2,355030801000003,"POLYGON ((340010.027 7392443.461, 339999.011 7...",355030801000003,189.0,189.0,189.0,0.0,0.0,187.0,1.0,1.0,0.0,0.0,189.0,0.0,0.0,0.0,False,False,high-high
3,355030801000004,"POLYGON ((340035.826 7392138.185, 339960.781 7...",355030801000004,181.0,181.0,181.0,0.0,0.0,181.0,0.0,0.0,0.0,0.0,181.0,0.0,0.0,0.0,False,False,non-significant
4,355030801000005,"POLYGON ((339655.092 7392153.506, 339647.532 7...",355030801000005,240.0,240.0,240.0,0.0,0.0,240.0,0.0,0.0,0.0,0.0,240.0,0.0,0.0,0.0,False,False,non-significant
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18948,355030896000243,"POLYGON ((358263.250 7395587.106, 358263.354 7...",355030896000243,96.0,96.0,96.0,0.0,0.0,96.0,0.0,0.0,0.0,0.0,96.0,0.0,0.0,0.0,False,False,non-significant
18949,355030896000244,"POLYGON ((358308.049 7395452.215, 358307.823 7...",355030896000244,54.0,54.0,54.0,0.0,0.0,52.0,0.0,0.0,0.0,2.0,54.0,0.0,0.0,0.0,False,False,non-significant
18950,355030896000245,"POLYGON ((354654.416 7398105.552, 354676.451 7...",355030896000245,87.0,87.0,87.0,0.0,0.0,15.0,0.0,0.0,0.0,72.0,87.0,0.0,0.0,0.0,False,False,non-significant
18951,355030896000246,"POLYGON ((354778.438 7398059.051, 354812.268 7...",355030896000246,38.0,38.0,38.0,0.0,0.0,18.0,0.0,0.0,0.0,20.0,38.0,0.0,0.0,0.0,False,False,non-significant


In [49]:
gdf_setor_lisa[sample_filter]

,cd_original_setor_censitario,geometry,Cod_setor,Domicilio01_SP1_V001,Domicilio01_SP1_V002,Domicilio01_SP1_V012,Domicilio01_SP1_V013,Domicilio01_SP1_V014,Domicilio01_SP1_V017,Domicilio01_SP1_V018,Domicilio01_SP1_V019,Domicilio01_SP1_V020,Domicilio01_SP1_V021,Domicilio01_SP1_V035,Domicilio01_SP1_V039,Domicilio01_SP1_V040,Domicilio01_SP1_V041,missing,supressed,lisa_class
2,355030801000003,"POLYGON ((340010.027 7392443.461, 339999.011 7...",355030801000003,189.0,189.0,189.0,0.0,0.0,187.0,1.0,1.0,0.0,0.0,189.0,0.0,0.0,0.0,False,False,high-high
19,355030801000020,"POLYGON ((340088.447 7391835.084, 340084.309 7...",355030801000020,249.0,249.0,249.0,0.0,0.0,245.0,0.0,1.0,0.0,2.0,249.0,0.0,0.0,0.0,False,False,high-high
44,355030801000045,"POLYGON ((339265.785 7391602.236, 339195.036 7...",355030801000045,113.0,113.0,112.0,0.0,0.0,105.0,4.0,2.0,0.0,1.0,113.0,0.0,0.0,0.0,False,False,high-high
47,355030801000048,"POLYGON ((338723.577 7392061.306, 338712.262 7...",355030801000048,138.0,138.0,138.0,0.0,0.0,138.0,0.0,0.0,0.0,0.0,138.0,0.0,0.0,0.0,False,False,high-high
62,355030801000063,"POLYGON ((339992.569 7393535.485, 340002.893 7...",355030801000063,126.0,126.0,126.0,0.0,0.0,126.0,0.0,0.0,0.0,0.0,126.0,0.0,0.0,0.0,False,False,high-high
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18912,355030896000207,"POLYGON ((357068.601 7395859.545, 357029.165 7...",355030896000207,182.0,182.0,182.0,0.0,0.0,181.0,1.0,0.0,0.0,0.0,182.0,0.0,0.0,0.0,False,False,high-high
18914,355030896000209,"POLYGON ((355336.270 7396003.885, 355249.431 7...",355030896000209,129.0,129.0,128.0,0.0,0.0,127.0,1.0,1.0,0.0,0.0,129.0,0.0,0.0,0.0,False,False,high-high
18922,355030896000217,"POLYGON ((354828.555 7396556.448, 354825.334 7...",355030896000217,127.0,127.0,127.0,0.0,0.0,126.0,1.0,0.0,0.0,0.0,127.0,0.0,0.0,0.0,False,False,high-high
18931,355030896000226,"POLYGON ((354485.510 7398198.671, 354488.078 7...",355030896000226,193.0,193.0,193.0,0.0,0.0,66.0,127.0,0.0,0.0,0.0,193.0,0.0,0.0,0.0,False,False,high-high


In [51]:
# TODO: repetir a mesma estrutura de maneira iterativa para todas as classificações

## Selecionar aleatoriamente um distrito para a exibição (alterando a caixa de seleção)

In [52]:
gdf_setor_distrito = load_parquet('intersection_setor_distrito_municipal')
gdf_setor_distrito

,geometry,cd_original_setor_censitario,negative_buffer,cd_identificador_distrito,cd_distrito_municipal,cd_identificador_subprefeitura,nm_distrito_municipal,sg_distrito_municipal,right_geometry,intersection,intersection_pct
0,"POLYGON ((339816.876 7392593.185, 339762.085 7...",355030801000001,"POLYGON ((339795.314 7392956.582, 339795.357 7...",8583441.0,1,25.0,AGUA RASA,ARA,"POLYGON ((341093.380 7391912.491, 341093.329 7...","POLYGON ((339816.876 7392593.185, 339762.085 7...",1.000000
1,"POLYGON ((339898.791 7392663.965, 339890.991 7...",355030801000002,"POLYGON ((339899.542 7392692.981, 339897.869 7...",8583441.0,1,25.0,AGUA RASA,ARA,"POLYGON ((341093.380 7391912.491, 341093.329 7...","POLYGON ((339898.791 7392663.965, 339890.991 7...",1.000000
2,"POLYGON ((340010.027 7392443.461, 339999.011 7...",355030801000003,"POLYGON ((340000.943 7392448.702, 339993.107 7...",8583441.0,1,25.0,AGUA RASA,ARA,"POLYGON ((341093.380 7391912.491, 341093.329 7...","POLYGON ((340010.027 7392443.461, 339999.011 7...",1.000000
3,"POLYGON ((340035.826 7392138.185, 339960.781 7...",355030801000004,"POLYGON ((339816.824 7392915.290, 339817.007 7...",8583441.0,1,25.0,AGUA RASA,ARA,"POLYGON ((341093.380 7391912.491, 341093.329 7...","POLYGON ((340035.826 7392138.185, 339960.781 7...",1.000000
4,"POLYGON ((339655.092 7392153.506, 339647.532 7...",355030801000005,"POLYGON ((339634.622 7392191.980, 339659.950 7...",8583441.0,1,25.0,AGUA RASA,ARA,"POLYGON ((341093.380 7391912.491, 341093.329 7...","POLYGON ((339655.092 7392153.506, 339647.532 7...",1.000000
...,...,...,...,...,...,...,...,...,...,...,...
18948,"POLYGON ((358263.250 7395587.106, 358263.354 7...",355030896000243,"POLYGON ((358186.206 7395515.305, 358193.682 7...",8583486.0,96,28.0,LAJEADO,LAJ,"POLYGON ((357579.815 7394795.986, 357572.551 7...","POLYGON ((358269.397 7395437.334, 358263.398 7...",0.927643
18949,"POLYGON ((358308.049 7395452.215, 358307.823 7...",355030896000244,"POLYGON ((358298.766 7395439.673, 358298.783 7...",8583486.0,96,28.0,LAJEADO,LAJ,"POLYGON ((357579.815 7394795.986, 357572.551 7...","POLYGON ((358345.910 7395353.418, 358345.564 7...",0.961410
18950,"POLYGON ((354654.416 7398105.552, 354676.451 7...",355030896000245,"POLYGON ((354660.086 7398092.819, 354673.006 7...",8583486.0,96,28.0,LAJEADO,LAJ,"POLYGON ((357579.815 7394795.986, 357572.551 7...","POLYGON ((354654.416 7398105.552, 354676.451 7...",1.000000
18951,"POLYGON ((354778.438 7398059.051, 354812.268 7...",355030896000246,"POLYGON ((354784.000 7398046.123, 354808.612 7...",8583486.0,96,28.0,LAJEADO,LAJ,"POLYGON ((357579.815 7394795.986, 357572.551 7...","POLYGON ((354778.438 7398059.051, 354812.268 7...",1.000000


In [62]:
gdf_distrito = gdf_setor_distrito[['cd_identificador_distrito', 'cd_distrito_municipal', 'nm_distrito_municipal', 'sg_distrito_municipal', 'right_geometry']].copy()
gdf_distrito = gdf_distrito.drop_duplicates()
gdf_distrito = gdf_distrito.rename(columns={'right_geometry': 'geometry'})
gdf_distrito

,cd_identificador_distrito,cd_distrito_municipal,nm_distrito_municipal,sg_distrito_municipal,geometry
0,8583441.0,1,AGUA RASA,ARA,"POLYGON ((341093.380 7391912.491, 341093.329 7..."
130,8583442.0,2,ALTO DE PINHEIROS,API,"POLYGON ((326739.457 7395585.840, 326741.473 7..."
209,8583443.0,3,ANHANGUERA,ANH,"POLYGON ((318695.858 7403032.323, 318677.730 7..."
330,8583444.0,4,ARICANDUVA,ARI,"POLYGON ((344068.781 7390960.855, 344068.149 7..."
456,8583445.0,5,ARTUR ALVIM,AAL,"POLYGON ((349416.226 7397718.808, 349416.884 7..."
...,...,...,...,...,...
17928,8583436.0,92,VILA MEDEIROS,VMD,"POLYGON ((340579.417 7401672.553, 340579.493 7..."
18149,8583437.0,93,VILA PRUDENTE,VPR,"POLYGON ((341731.295 7388459.495, 341723.755 7..."
18311,8583440.0,94,VILA SONIA,VSO,"POLYGON ((320805.525 7390280.495, 320824.353 7..."
18581,8583415.0,95,SAO DOMINGOS,SDO,"POLYGON ((319854.036 7402894.086, 319864.990 7..."


In [58]:
nm_distrito_selecionado = gdf_distrito.sample(n=1).iloc[0,2]
nm_distrito_selecionado

'PARI'

## Centraliza o mapa no distrito

In [72]:
# CartoDB Positron
map = folium.Map(tiles="CartoDB Positron", zoom_start=9)
map

In [74]:
# Para facilitar a edição, defino a tolerância de simplificação das geometrias
simpification_tolerance = 10**-6


Como o folium trabalha apenas na projeção `EPSG4326`, criarei um novo geodataframe convertido para utilizar nos próximos mapas

In [75]:
fol_df = gdf_distrito.copy().to_crs(epsg=4326)
fol_df

,cd_identificador_distrito,cd_distrito_municipal,nm_distrito_municipal,sg_distrito_municipal,geometry
0,8583441.0,1,AGUA RASA,ARA,"POLYGON ((-46.55714 -23.57545, -46.55714 -23.5..."
130,8583442.0,2,ALTO DE PINHEIROS,API,"POLYGON ((-46.69732 -23.54081, -46.69730 -23.5..."
209,8583443.0,3,ANHANGUERA,ANH,"POLYGON ((-46.77519 -23.47270, -46.77536 -23.4..."
330,8583444.0,4,ARICANDUVA,ARI,"POLYGON ((-46.52809 -23.58433, -46.52809 -23.5..."
456,8583445.0,5,ARTUR ALVIM,AAL,"POLYGON ((-46.47502 -23.52381, -46.47501 -23.5..."
...,...,...,...,...,...
17928,8583436.0,92,VILA MEDEIROS,VMD,"POLYGON ((-46.56113 -23.48727, -46.56113 -23.4..."
18149,8583437.0,93,VILA PRUDENTE,VPR,"POLYGON ((-46.55125 -23.60669, -46.55133 -23.6..."
18311,8583440.0,94,VILA SONIA,VSO,"POLYGON ((-46.75607 -23.58806, -46.75588 -23.5..."
18581,8583415.0,95,SAO DOMINGOS,SDO,"POLYGON ((-46.76387 -23.47407, -46.76376 -23.4..."


Primeiro, adicionarei o contorno geral dos setores censitários para servir de base

In [76]:
fol_df = fol_df.loc[fol_df['nm_distrito_municipal']==nm_distrito_selecionado]
sim_geo = fol_df["geometry"].simplify(tolerance=simpification_tolerance)
geo_j = sim_geo.to_json()
geo_j = folium.GeoJson(data=geo_j, style_function=lambda x: {'color': 'navy', 'fillOpacity': 0.2})
geo_j.add_to(map)
map.fit_bounds(map.get_bounds())
map

## Exibir os polígonos dos setores censitários daquele distrito com cores baseadas nas categorias acima

In [77]:
fol_df_2 = gdf_setor.copy().to_crs(epsg=4326)
fol_df_2

,cd_original_setor_censitario,geometry,Cod_setor,Domicilio01_SP1_V001,Domicilio01_SP1_V002,Domicilio01_SP1_V012,Domicilio01_SP1_V013,Domicilio01_SP1_V014,Domicilio01_SP1_V017,Domicilio01_SP1_V018,Domicilio01_SP1_V019,Domicilio01_SP1_V020,Domicilio01_SP1_V021,Domicilio01_SP1_V035,Domicilio01_SP1_V039,Domicilio01_SP1_V040,Domicilio01_SP1_V041,missing,supressed
0,355030801000001,"POLYGON ((-46.56957 -23.56917, -46.57010 -23.5...",355030801000001,268.0,266.0,266.0,0.0,0.0,266.0,0.0,0.0,0.0,0.0,265.0,0.0,1.0,0.0,False,False
1,355030801000002,"POLYGON ((-46.56876 -23.56854, -46.56883 -23.5...",355030801000002,306.0,306.0,306.0,0.0,0.0,306.0,0.0,0.0,0.0,0.0,306.0,0.0,0.0,0.0,False,False
2,355030801000003,"POLYGON ((-46.56769 -23.57054, -46.56780 -23.5...",355030801000003,189.0,189.0,189.0,0.0,0.0,187.0,1.0,1.0,0.0,0.0,189.0,0.0,0.0,0.0,False,False
3,355030801000004,"POLYGON ((-46.56747 -23.57330, -46.56821 -23.5...",355030801000004,181.0,181.0,181.0,0.0,0.0,181.0,0.0,0.0,0.0,0.0,181.0,0.0,0.0,0.0,False,False
4,355030801000005,"POLYGON ((-46.57120 -23.57313, -46.57127 -23.5...",355030801000005,240.0,240.0,240.0,0.0,0.0,240.0,0.0,0.0,0.0,0.0,240.0,0.0,0.0,0.0,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18948,355030896000243,"POLYGON ((-46.38858 -23.54386, -46.38858 -23.5...",355030896000243,96.0,96.0,96.0,0.0,0.0,96.0,0.0,0.0,0.0,0.0,96.0,0.0,0.0,0.0,False,False
18949,355030896000244,"POLYGON ((-46.38815 -23.54508, -46.38816 -23.5...",355030896000244,54.0,54.0,54.0,0.0,0.0,52.0,0.0,0.0,0.0,2.0,54.0,0.0,0.0,0.0,False,False
18950,355030896000245,"POLYGON ((-46.42368 -23.52080, -46.42347 -23.5...",355030896000245,87.0,87.0,87.0,0.0,0.0,15.0,0.0,0.0,0.0,72.0,87.0,0.0,0.0,0.0,False,False
18951,355030896000246,"POLYGON ((-46.42247 -23.52123, -46.42214 -23.5...",355030896000246,38.0,38.0,38.0,0.0,0.0,18.0,0.0,0.0,0.0,20.0,38.0,0.0,0.0,0.0,False,False


In [78]:
gdf_setor_distrito_filtrado = gdf_setor_distrito[gdf_setor_distrito['nm_distrito_municipal']==nm_distrito_selecionado]
filter_setor = fol_df_2['cd_original_setor_censitario'].isin(gdf_setor_distrito_filtrado['cd_original_setor_censitario'])

fol_df_2 = fol_df_2.loc[filter_setor]
sim_geo = fol_df_2["geometry"].simplify(tolerance=simpification_tolerance)
geo_j = sim_geo.to_json()
geo_j = folium.GeoJson(data=geo_j, style_function=lambda x: {'color': 'orange', 'fillOpacity': 0.1})
geo_j.add_to(map)
map.fit_bounds(map.get_bounds())
map

### Segundo modo (.explore)

In [79]:
base_map = fol_df.explore(color='navy', tiles="CartoDB Positron")
m = fol_df_2.explore(color='orange', m=base_map)
m

## Exibir informações adicionais sobre o setor quando seu polígono for clicado no mapa

## Permitir a  a inclusão ou remoção de outras camadas relacionadas a um determinado setor, com base nos assets `intersection_setor_*`